In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from itertools import product
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error, r2_score
import numpy as np
import joblib
import first_analysis
import pandas as pd

In [ ]:
class FeedForward(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes, depth, activation_f):
        super(FeedForward, self).__init__()
        
        function = None

        if activation_f == 'ReLU':
            function = nn.ReLU()
        elif activation_f == 'Tanh':
            function = nn.Tanh()
        else:
            function = nn.Sigmoid()

        model = [
            nn.Linear(input_size, hidden_size),
            nn.BatchNorm1d(hidden_size),
            #nn.ReLU()
            function
        ]

        block = [
            nn.Linear(hidden_size, hidden_size),
            nn.BatchNorm1d(hidden_size),
            #nn.ReLU()
            function
        ]

        for i in range(depth):
            model += block

        
        self.model = nn.Sequential(*model)
        
        self.output = nn.Linear(hidden_size, num_classes)
        

    def forward(self, x):
        h = self.model(x)
        out = self.output(h)
        return out
    
def prepare_data(X_train, y_train, X_val, y_val):
    X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
    y_train_tensor = torch.tensor(y_train.values, dtype=torch.float32)
    X_val_tensor = torch.tensor(X_val, dtype=torch.float32)
    y_val_tensor = torch.tensor(y_val.values, dtype=torch.float32)

    train_dataset = torch.utils.data.TensorDataset(X_train_tensor, y_train_tensor)
    val_dataset = torch.utils.data.TensorDataset(X_val_tensor, y_val_tensor)

    return train_dataset, val_dataset

def train_model(model, train_loader, criterion, optimizer, num_epochs):
    model.train()
    for epoch in range(num_epochs):
        for inputs, labels in train_loader:
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels.unsqueeze(1))
            #loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

def evaluate_model(model, val_loader):
    model.eval()
    predictions = []
    targets = []
    with torch.no_grad():
        for inputs, labels in val_loader:
            outputs = model(inputs)
            predictions.extend(outputs.numpy())
            targets.extend(labels.numpy())
    predictions = np.array(predictions)
    targets = np.array(targets)
    mse = mean_squared_error(targets, predictions)
    mae = mean_absolute_error(targets, predictions)
    mape = mean_absolute_percentage_error(targets, predictions)
    r2 = r2_score(targets, predictions)
    return mse, mae, mape, r2

In [ ]:
def val_fnn(X_train, y_train, X_val, y_val, depth_values, hidden_size_values, lr_values, activation_fs):
    results = []

    train_dataset, val_dataset = prepare_data(X_train, y_train, X_val, y_val)
    train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=64, shuffle=True)
    val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=64, shuffle=False)
            
    input_size = X_train.shape[1]
    num_classes = 1

    for depth, hidden_size, lr, activation_f in product(depth_values, hidden_size_values, lr_values, activation_fs):
        #print(f"Testing depth={depth}, hidden_size={hidden_size}, lr={lr}")

        model = FeedForward(input_size, hidden_size, num_classes, depth, activation_f)

        criterion = nn.MSELoss()
        optimizer = optim.Adam(model.parameters(), lr=lr)

        num_epochs = 10
        train_model(model, train_loader, criterion, optimizer, num_epochs)

        mse, mae, mape, r2 = evaluate_model(model, val_loader)
        
        results.append({
            'depth': depth,
            'hidden_size': hidden_size,
            'lr': lr,
            'mse': mse,
            'mae': mae,
            'mape': mape,
            'r2': r2,
            'activation_f': activation_f
        })
    return results

In [ ]:
# Ricordati che probabilemte qui dovrai effettuare anche del dropout e della regolarizzazione. MA PRIMA STUDIALE

In [ ]:
# DA PROVARE CON DIVERSI DATASET
X_train_scaled = np.load("./Datasets/Train_Val_Test/Scaled/X_train_scaled.npy")
X_val_scaled = np.load("./Datasets/Train_Val_Test/Scaled/X_val_scaled.npy")
y_train, y_val = first_analysis.load_df("./Datasets/Train_Val_Test/y_train.csv", "./Datasets/Train_Val_Test/y_val.csv")

In [ ]:
# X_train_scaled = X_train_scaled[:200]
# X_val_scaled = X_val_scaled[:40]
# y_train = y_train.head(200)
# y_val = y_val.head(40)

In [ ]:
depth_values = list(range(0, 9))
hidden_size_values = [5, 10, 15, 20, 30, 50, 100, 200, 300, 400]
lr_values = [0.01, 0.1]
activation_fs = ['ReLU', 'Tanh', 'Sigmoid']

In [ ]:
results = val_fnn(X_train_scaled, y_train, X_val_scaled, y_val, depth_values, hidden_size_values, lr_values, activation_fs)

In [ ]:
df_res = pd.DataFrame(results)

In [ ]:
df_res.to_csv("./df_Res_fnn.csv")

In [ ]:
# Salva modello